In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm
from PIL import Image, ImageOps
from tensorflow.keras import models, layers, optimizers
# from tensorflow.keras import layers
# from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
# Creating a list of actors iterating through the folder of actors

test_dir = "/home/antoniosf/Desktop/Project_practice/20_actors"
actors = []
for filename in os.listdir(test_dir):
    actors.append(filename)
# actors

FileNotFoundError: [Errno 2] No such file or directory: '/home/antoniosf/Desktop/Project_practice/20_actors'

In [ ]:
# Creating a list of numbers to use as labels for each actor

actors.sort()

labels_values = []
for i in range(len(actors)):
    labels_values.append(i)
# labels_values

In [4]:
# Combining the two list into a dictionary

labels_dict = dict(zip(actors, labels_values))
# labels_dict

In [4]:
# Function to read the images

def read_images(directory):


    images = []
    labels = []
    for filename in os.listdir(directory):

        files = glob.glob(directory + "/" + filename + "/*.jpg")
        for f in tqdm(files):
            try:
                im = Image.open(f)
#                 im = ImageOps.grayscale(im2)
#                 im = im.resize(resize_to)
                im = np.array(im) / 255.0 # Converting to numpy array and normalizing
                im = im.astype("float32") # Converting to float
                im = tf.expand_dims(im, axis=-1) # Adding one dimension 
                images.append(im)
                name = filename.split("/")[-1].split(".")[0]
                label = labels_dict[name]
                labels.append(label)
            except Exception as e:
                print(f)
                print(e)

    return images, np.array(labels)



In [5]:
directory = "/Users/elias/Desktop/preprocessed test images"

X, y = read_images(directory=directory) #Loading the preprocessed images

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [7]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts)) # Checking the number of images per label

{0: 100,
 1: 100,
 2: 100,
 3: 100,
 4: 100,
 5: 100,
 6: 100,
 7: 100,
 8: 100,
 9: 100,
 10: 100,
 11: 100,
 12: 100,
 13: 100,
 14: 100,
 15: 100,
 16: 100,
 17: 100,
 18: 100,
 19: 100}

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [9]:
np.max(X_train), np.min(X_train) # Checking if normalized

(1.0, 0.0)

In [10]:
np.max(X_test), np.min(X_test) # Checking if normalized

(1.0, 0.0)

In [11]:
# Converting y to categorical

y_train = to_categorical(y_train, 20)
y_test = to_categorical(y_test, 20)

In [12]:
def initialize_model():
    model = models.Sequential()
#     model.add(layers.Conv2D(32, kernel_size=3, padding='same', input_shape=(128, 128, 1), activation='relu'))
    model.add(layers.Conv2D(16, (4,4), input_shape=(100, 100, 1)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

#     model.add(layers.Conv2D(64, (2, 2), padding='same', activation='relu'))
#     model.add(layers.MaxPooling2D(pool_size=(2, 2)))

#     model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
#     model.add(layers.MaxPooling2D(pool_size=(3, 3)))
    
#     model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
#     model.add(layers.MaxPooling2D(pool_size=(3, 3)))
    
    model.add(layers.Flatten())

#     model.add(layers.Dense(120, activation='relu'))
#     model.add(layers.Dense(60, activation='relu'))
#     model.add(layers.Dense(60, activation='relu'))


    model.add(layers.Dense(20, activation='softmax'))
    
    return model

In [13]:
model = initialize_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 97, 97, 16)        272       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 20)                737300    
Total params: 737,572
Trainable params: 737,572
Non-trainable params: 0
_________________________________________________________________


In [14]:
def compile_model(model):
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [15]:
# from tensorflow.keras.callbacks import EarlyStopping

model = initialize_model()
model = compile_model(model)

es = EarlyStopping(patience=10, verbose=1)

history = model.fit(np.array(X_train), y_train, 
                    validation_split=0.3,
                    callbacks=[es], 
                    epochs=100, 
                    batch_size=16)

Epoch 1/100
59/59 [==============================] - 4s 47ms/step - loss: 4.8416 - accuracy: 0.0691 - val_loss: 2.8315 - val_accuracy: 0.1687
Epoch 2/100
59/59 [==============================] - 2s 28ms/step - loss: 2.5176 - accuracy: 0.3516 - val_loss: 2.6110 - val_accuracy: 0.2903
Epoch 3/100
59/59 [==============================] - 2s 28ms/step - loss: 2.1069 - accuracy: 0.4690 - val_loss: 2.4103 - val_accuracy: 0.3474
Epoch 4/100
59/59 [==============================] - 2s 28ms/step - loss: 1.6158 - accuracy: 0.6388 - val_loss: 2.2638 - val_accuracy: 0.3921
Epoch 5/100
59/59 [==============================] - 2s 28ms/step - loss: 1.2811 - accuracy: 0.7546 - val_loss: 2.1668 - val_accuracy: 0.4144
Epoch 6/100
59/59 [==============================] - 2s 28ms/step - loss: 0.9813 - accuracy: 0.8341 - val_loss: 2.1910 - val_accuracy: 0.3896
Epoch 7/100
59/59 [==============================] - 2s 28ms/step - loss: 0.7959 - accuracy: 0.8548 - val_loss: 2.1006 - val_accuracy: 0.4144
Epoch 

In [16]:
model.evaluate(np.array(X_test), y_test, verbose=0)

[2.550947666168213, 0.42575758695602417]